In [78]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import re
import sklearn
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize
import string
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from sklearn.pipeline import Pipeline

In [14]:
df = pd.read_csv('data/train.csv')

In [15]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [16]:
df.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [17]:
df[(df['severe_toxic'] == 1) & (df['obscene'] == 0) & (df['threat'] == 0) & (df['insult'] == 0) & (df['identity_hate'] == 0)].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
4894,0cf5e967ded97d37,so everytime i reset my modem my ip changes\n\...,1,1,0,0,0,0
5610,0efdedac6cc0a8db,hello\nmother fucker hijo edputa,1,1,0,0,0,0
8326,16235f5ddce28a61,Paul Tibbit is a fucking-ass little piece of s...,1,1,0,0,0,0
12861,2215d63992760558,"NOT REALLY, GO AHEAD AND FUCK OFF BITCH!!!",1,1,0,0,0,0
18147,2fefda078c74afb3,Your a dick head!! \n\nHow dare you go around ...,1,1,0,0,0,0


In [18]:
df.drop(['id', 'toxic'], axis=1, inplace=True)

In [19]:
df.describe()

,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   severe_toxic   159571 non-null  int64 
 2   obscene        159571 non-null  int64 
 3   threat         159571 non-null  int64 
 4   insult         159571 non-null  int64 
 5   identity_hate  159571 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 7.3+ MB


In [21]:
df.isna().sum()

comment_text     0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [24]:
def conditions(dataFrame):
    if dataFrame['severe_toxic'] == 1 or dataFrame['obscene'] == 1 or dataFrame['threat'] == 1 or dataFrame['insult'] == 1 or dataFrame['identity_hate'] == 1:
        val = 1
    else:
        val = 0
    return val


df['general_toxic']= df.apply(conditions, axis=1)
df.head()

,comment_text,severe_toxic,obscene,threat,insult,identity_hate,general_toxic
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [25]:
snowball = SnowballStemmer(language="english")
import re
def preprocessig_text(text):
    stop_words = set(stopwords.words('english')) 
    text = text.lower()
    text = re.sub("[^a-zA-Z]", " ", text)
    text = word_tokenize(text)
    text = [i for i in text if i not in stop_words]
    text = [snowball.stem(i) for i in text]
    
   
    return ' '.join(text)

In [26]:
preprocess_text = df['comment_text'].apply(preprocessig_text)

In [27]:
 preprocess_text

0         explan edit made usernam hardcor metallica fan...
1         aww match background colour seem stuck thank t...
2         hey man realli tri edit war guy constant remov...
3         make real suggest improv wonder section statis...
4                                sir hero chanc rememb page
                                ...                        
159566    second time ask view complet contradict covera...
159567                    asham horribl thing put talk page
159568    spitzer umm there actual articl prostitut ring...
159569    look like actual put speedi first version dele...
159570    realli think understand came idea bad right aw...
Name: comment_text, Length: 159571, dtype: object

In [31]:
X = preprocess_text
y = df['general_toxic']

In [32]:
clf = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', LogisticRegression())
])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [36]:
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print('Testing accuracy is {}'.format(score))


Testing accuracy is 0.968541438195206


In [73]:
import pickle

In [74]:
with open('trained_model-0.1.0.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [76]:
nltk.__version__

'3.7'

In [79]:
sklearn.__version__

'1.2.1'